process: 

for every precipitation file, calculate average precip per hour for the appropriate domain

calculate the sum across morning, afternoon, evening precipitation

categorize each day as an "APE" or not 

there should be: "368 APEs from the 2172 soundings" or about 16% of APEs

notes: 
"For each grid point, the amount of precipitation in the previous hour is given" (precip metadata)

In [1]:
import pandas as pd
from func import cal_buoyancy,sounding_cal
import glob
import os
from haversine import haversine, Unit
import re
import datetime
import metpy.calc as mpcalc
from metpy.units import units
from datetime import datetime, timedelta
from metpy import constants
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from pyhdf import SD
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
import seaborn as sns
import tropycal.tracks as tracks
from pydoc import help
from pyhdf.SD import *
from sklearn.preprocessing import QuantileTransformer
import xarray as xr
import netCDF4 as nc
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk
from io import StringIO 
import requests
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
# Set display options to show all rows and columns in dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
hurdat = tracks.TrackDataset(basin='north_atlantic',source='hurdat',include_btk=False)

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (2.87 seconds)


In [4]:
#read in precipitation files:
#2001-2019 warm-season

directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/abrfc_final"

#files either .cdf or .nc
#file name looks like: sgpabrfcprecipX1.c1.19971231.000000.custom.cdf
#file name looks like: sgpabrfcprecipX1.c1.20171231.000000.custom.nc

files = [
    os.path.join(directory_path, file)
    for file in os.listdir(directory_path)
    if file.endswith(".cdf") or file.endswith(".nc")
]

filtered_files = []
for file in files:
    file_name = os.path.basename(file)
    year_month = file_name.split('.')[2]
    if len(year_month) == 8:
        year = int(year_month[0:4])
        month = int(year_month[4:6])
        day = int(year_month[6:8])
        hours_mins_seconds = file_name.split('.')[3]
        hour = int(hours_mins_seconds[0:2])
        if 5 <= month <= 9 and 2020 > year > 2000:
            filtered_files.append(file)

precipdfs = []
for ds in sorted(filtered_files):
    ds = xr.open_dataset(ds)
    precipdfs.append(ds)
    
len(precipdfs)

2906

In [96]:
with open('gprecip.pdkl', 'rb') as file:
    
    gprecip = pickle.load(file)

gprecip.shape

(2752, 1)

In [104]:
gtuples = []

for i in range(gprecip.shape[0]):
    
    date = gprecip.index[i]
    
    precip = np.array(gprecip['p'][i])
    
    if len(precip) == 24:
        
        #corresponds to hours 6-13
        precip_morning = sum(precip[6:14])
        
        #corresponds to hours 14-20
        precip_afternoon = sum(precip[14:21]) 
    
        #corresponds to hours 21-24
        precip_evening = sum(precip[21:24])
        
        peak_hr = np.argmax(precip)
        
        peak_amount = precip[peak_hr]
        
        tup = (date, precip_morning, precip_afternoon, precip_evening, peak_hr, peak_amount)
        
        gtuples.append(tup)

gaperes = pd.DataFrame(gtuples,columns=['date','morning','afternoon','evening','peak_hr', 'peak_amount'])

In [109]:
conditions = (gaperes['afternoon'] > gaperes['evening']) & \
             (gaperes['afternoon'] > 2 * gaperes['morning']) & \
             (gaperes['peak_hr'].isin([14, 15, 16, 17, 18, 19, 20]))

gaperes['APE'] = conditions

gpdf = gaperes[['date','APE','afternoon']]


gpdf.set_index('date', inplace=True)


np.sum(gpdf['APE'])

356

In [20]:
# annie -- hourly precipitation calculation

# cleanp = []

# for df in precipdfs:
    
#     min_p = 0
    
#     #equivalent to 250mm of rain per hour
    
#     max_p = 25000

#     precip = df.variables['precip'].data
          
#     mask = ((precip < min_p) | (precip > max_p))

#     precip_clean = np.where(mask, np.nan, precip)
    
#     nan_count = np.count_nonzero(np.isnan(precip_clean))

#     if nan_count == 0:
        
#         cleanp.append(df)
        

# print(len(cleanp))

2898


In [21]:
# annie -- hourly precipitation calculation

# sgp = (36.607322, -97.487643)

# max_distance = 50.0  # kilometers

# precip_pairs = []

# precip_index_of_pairs = []

# for i in range(160):

#     for j in range(336):

#         lat = cleanp[0].variables["lat"].data[i, j]

#         lon = cleanp[0].variables["lon"].data[i, j]

#         lat_lon_pair = (lat,lon)

#         distance = haversine(sgp, lat_lon_pair, unit='km')

#         if distance <= max_distance:

#             pair = (lat, lon)

#             index = (i, j)

#             precip_pairs.append(pair)

#             precip_index_of_pairs.append(index)

# len(precip_index_of_pairs)

472

In [22]:
#annie -- hourly precipitation calculation 

# rewritting to shift to oklahoma time

# hourlyp = []

# count=0

# for df in cleanp:
    
#     hrs = df.variables['precip'].shape[0]
    
#     for hr in range(hrs):
        
#         ptotal = 0
        
#         time = df['time_offset'].data[hr]
        
#         lst = time - pd.to_timedelta('6 hours')
    
#         for index in precip_index_of_pairs:
          
#             p = df.variables['precip'].data[hr,index[0],index[1]]
            
#             #convert to mm
            
#             p = p * 0.01
                
#             ptotal += p
    
#         avgp = ptotal/len(precip_index_of_pairs)
    
#         tup = (lst, avgp)
        
#         hourlyp.append(tup)
    
#     count+=1
#     print(count/2898, "% complete")


0.0003450655624568668 % complete
0.0006901311249137336 % complete
0.0010351966873706005 % complete
0.0013802622498274672 % complete
0.001725327812284334 % complete
0.002070393374741201 % complete
0.0024154589371980675 % complete
0.0027605244996549345 % complete
0.003105590062111801 % complete
0.003450655624568668 % complete
0.003795721187025535 % complete
0.004140786749482402 % complete
0.004485852311939268 % complete
0.004830917874396135 % complete
0.005175983436853002 % complete
0.005521048999309869 % complete
0.005866114561766736 % complete
0.006211180124223602 % complete
0.006556245686680469 % complete
0.006901311249137336 % complete
0.007246376811594203 % complete
0.00759144237405107 % complete
0.007936507936507936 % complete
0.008281573498964804 % complete
0.00862663906142167 % complete
0.008971704623878536 % complete
0.009316770186335404 % complete
0.00966183574879227 % complete
0.010006901311249138 % complete
0.010351966873706004 % complete
0.01069703243616287 % complete
0.0110

0.09075224292615597 % complete
0.09109730848861283 % complete
0.0914423740510697 % complete
0.09178743961352658 % complete
0.09213250517598344 % complete
0.0924775707384403 % complete
0.09282263630089717 % complete
0.09316770186335403 % complete
0.09351276742581091 % complete
0.09385783298826778 % complete
0.09420289855072464 % complete
0.0945479641131815 % complete
0.09489302967563837 % complete
0.09523809523809523 % complete
0.09558316080055211 % complete
0.09592822636300898 % complete
0.09627329192546584 % complete
0.0966183574879227 % complete
0.09696342305037957 % complete
0.09730848861283643 % complete
0.09765355417529331 % complete
0.09799861973775018 % complete
0.09834368530020704 % complete
0.0986887508626639 % complete
0.09903381642512077 % complete
0.09937888198757763 % complete
0.09972394755003451 % complete
0.10006901311249138 % complete
0.10041407867494824 % complete
0.1007591442374051 % complete
0.10110420979986197 % complete
0.10144927536231885 % complete
0.101794340924

0.1828847481021394 % complete
0.18322981366459629 % complete
0.18357487922705315 % complete
0.18391994478951001 % complete
0.18426501035196688 % complete
0.18461007591442374 % complete
0.1849551414768806 % complete
0.18530020703933747 % complete
0.18564527260179434 % complete
0.1859903381642512 % complete
0.18633540372670807 % complete
0.18668046928916493 % complete
0.18702553485162182 % complete
0.1873706004140787 % complete
0.18771566597653555 % complete
0.18806073153899242 % complete
0.18840579710144928 % complete
0.18875086266390614 % complete
0.189095928226363 % complete
0.18944099378881987 % complete
0.18978605935127674 % complete
0.1901311249137336 % complete
0.19047619047619047 % complete
0.19082125603864733 % complete
0.19116632160110422 % complete
0.1915113871635611 % complete
0.19185645272601795 % complete
0.19220151828847482 % complete
0.19254658385093168 % complete
0.19289164941338854 % complete
0.1932367149758454 % complete
0.19358178053830227 % complete
0.193926846100759

0.2753623188405797 % complete
0.2757073844030366 % complete
0.27605244996549344 % complete
0.27639751552795033 % complete
0.27674258109040717 % complete
0.27708764665286406 % complete
0.2774327122153209 % complete
0.2777777777777778 % complete
0.2781228433402346 % complete
0.2784679089026915 % complete
0.27881297446514836 % complete
0.27915804002760525 % complete
0.2795031055900621 % complete
0.279848171152519 % complete
0.28019323671497587 % complete
0.2805383022774327 % complete
0.2808833678398896 % complete
0.28122843340234643 % complete
0.2815734989648033 % complete
0.28191856452726016 % complete
0.28226363008971705 % complete
0.2826086956521739 % complete
0.2829537612146308 % complete
0.2832988267770876 % complete
0.2836438923395445 % complete
0.2839889579020014 % complete
0.28433402346445824 % complete
0.28467908902691513 % complete
0.28502415458937197 % complete
0.28536922015182886 % complete
0.2857142857142857 % complete
0.2860593512767426 % complete
0.2864044168391994 % comple

0.3688750862663906 % complete
0.3692201518288475 % complete
0.3695652173913043 % complete
0.3699102829537612 % complete
0.3702553485162181 % complete
0.37060041407867494 % complete
0.37094547964113184 % complete
0.3712905452035887 % complete
0.37163561076604557 % complete
0.3719806763285024 % complete
0.3723257418909593 % complete
0.37267080745341613 % complete
0.373015873015873 % complete
0.37336093857832986 % complete
0.37370600414078675 % complete
0.37405106970324364 % complete
0.3743961352657005 % complete
0.3747412008281574 % complete
0.3750862663906142 % complete
0.3754313319530711 % complete
0.37577639751552794 % complete
0.37612146307798483 % complete
0.37646652864044167 % complete
0.37681159420289856 % complete
0.3771566597653554 % complete
0.3775017253278123 % complete
0.3778467908902691 % complete
0.378191856452726 % complete
0.3785369220151829 % complete
0.37888198757763975 % complete
0.37922705314009664 % complete
0.3795721187025535 % complete
0.37991718426501037 % complet

...

0.5565907522429262 % complete
0.556935817805383 % complete
0.5572808833678399 % complete
0.5576259489302967 % complete
0.5579710144927537 % complete
0.5583160800552105 % complete
0.5586611456176673 % complete
0.5590062111801242 % complete
0.5593512767425811 % complete
0.559696342305038 % complete
0.5600414078674948 % complete
0.5603864734299517 % complete
0.5607315389924086 % complete
0.5610766045548654 % complete
0.5614216701173222 % complete
0.5617667356797792 % complete
0.562111801242236 % complete
0.5624568668046929 % complete
0.5628019323671497 % complete
0.5631469979296067 % complete
0.5634920634920635 % complete
0.5638371290545203 % complete
0.5641821946169773 % complete
0.5645272601794341 % complete
0.564872325741891 % complete
0.5652173913043478 % complete
0.5655624568668047 % complete
0.5659075224292616 % complete
0.5662525879917184 % complete
0.5665976535541752 % complete
0.5669427191166322 % complete
0.567287784679089 % complete
0.5676328502415459 % complete
0.5679779158040

0.6514837819185645 % complete
0.6518288474810214 % complete
0.6521739130434783 % complete
0.6525189786059351 % complete
0.6528640441683919 % complete
0.6532091097308489 % complete
0.6535541752933057 % complete
0.6538992408557626 % complete
0.6542443064182195 % complete
0.6545893719806763 % complete
0.6549344375431332 % complete
0.65527950310559 % complete
0.655624568668047 % complete
0.6559696342305038 % complete
0.6563146997929606 % complete
0.6566597653554175 % complete
0.6570048309178744 % complete
0.6573498964803313 % complete
0.6576949620427881 % complete
0.658040027605245 % complete
0.6583850931677019 % complete
0.6587301587301587 % complete
0.6590752242926156 % complete
0.6594202898550725 % complete
0.6597653554175293 % complete
0.6601104209799862 % complete
0.660455486542443 % complete
0.6608005521049 % complete
0.6611456176673568 % complete
0.6614906832298136 % complete
0.6618357487922706 % complete
0.6621808143547274 % complete
0.6625258799171843 % complete
0.6628709454796411

0.7463768115942029 % complete
0.7467218771566597 % complete
0.7470669427191167 % complete
0.7474120082815735 % complete
0.7477570738440303 % complete
0.7481021394064873 % complete
0.7484472049689441 % complete
0.748792270531401 % complete
0.7491373360938578 % complete
0.7494824016563147 % complete
0.7498274672187716 % complete
0.7501725327812284 % complete
0.7505175983436853 % complete
0.7508626639061422 % complete
0.751207729468599 % complete
0.7515527950310559 % complete
0.7518978605935127 % complete
0.7522429261559697 % complete
0.7525879917184265 % complete
0.7529330572808833 % complete
0.7532781228433403 % complete
0.7536231884057971 % complete
0.753968253968254 % complete
0.7543133195307108 % complete
0.7546583850931677 % complete
0.7550034506556246 % complete
0.7553485162180814 % complete
0.7556935817805382 % complete
0.7560386473429952 % complete
0.756383712905452 % complete
0.7567287784679089 % complete
0.7570738440303658 % complete
0.7574189095928227 % complete
0.757763975155

0.8412698412698413 % complete
0.8416149068322981 % complete
0.841959972394755 % complete
0.8423050379572119 % complete
0.8426501035196687 % complete
0.8429951690821256 % complete
0.8433402346445825 % complete
0.8436853002070394 % complete
0.8440303657694962 % complete
0.844375431331953 % complete
0.84472049689441 % complete
0.8450655624568668 % complete
0.8454106280193237 % complete
0.8457556935817805 % complete
0.8461007591442374 % complete
0.8464458247066943 % complete
0.8467908902691511 % complete
0.8471359558316081 % complete
0.8474810213940649 % complete
0.8478260869565217 % complete
0.8481711525189786 % complete
0.8485162180814355 % complete
0.8488612836438924 % complete
0.8492063492063492 % complete
0.849551414768806 % complete
0.849896480331263 % complete
0.8502415458937198 % complete
0.8505866114561766 % complete
0.8509316770186336 % complete
0.8512767425810904 % complete
0.8516218081435473 % complete
0.8519668737060041 % complete
0.852311939268461 % complete
0.852657004830917

0.9365079365079365 % complete
0.9368530020703933 % complete
0.9371980676328503 % complete
0.9375431331953071 % complete
0.937888198757764 % complete
0.9382332643202208 % complete
0.9385783298826778 % complete
0.9389233954451346 % complete
0.9392684610075914 % complete
0.9396135265700483 % complete
0.9399585921325052 % complete
0.940303657694962 % complete
0.9406487232574189 % complete
0.9409937888198758 % complete
0.9413388543823327 % complete
0.9416839199447895 % complete
0.9420289855072463 % complete
0.9423740510697033 % complete
0.9427191166321601 % complete
0.943064182194617 % complete
0.9434092477570738 % complete
0.9437543133195307 % complete
0.9440993788819876 % complete
0.9444444444444444 % complete
0.9447895100069013 % complete
0.9451345755693582 % complete
0.945479641131815 % complete
0.9458247066942719 % complete
0.9461697722567288 % complete
0.9465148378191857 % complete
0.9468599033816425 % complete
0.9472049689440993 % complete
0.9475500345065563 % complete
0.947895100069

In [58]:
# annie -- hourly precipitation calculation

# pdf = pd.DataFrame(hourlyp, columns=['time','p'])

# pdf['timestamp'] = pd.to_datetime(pdf['time'])

# pdf['date'] = pdf['timestamp'].dt.date

# res = pdf.groupby('date')['p'].agg(list).reset_index()

# res.shape

(2925, 2)

In [59]:
# tuples = []

# for i in range(res.shape[0]):
    
#     date = res['date'][i]
    
#     precip = np.array(res['p'][i])
    
#     if len(precip) == 24:
        
#         #corresponds to hours 6-13
#         precip_morning = sum(precip[7:15])
        
#         #corresponds to hours 14-20
#         precip_afternoon = sum(precip[15:22]) 
    
#         #corresponds to hours 21-24
#         precip_evening = sum(precip[22:24])
        
#         peak_hr = np.argmax(precip)
        
#         peak_amount = precip[peak_hr]
        
#         tup = (date, precip_morning, precip_afternoon, precip_evening, peak_hr, peak_amount)
        
#         tuples.append(tup)

# aperes = pd.DataFrame(tuples,columns=['date','morning','afternoon','evening','peak_hr', 'peak_amount'])

In [83]:
# annie

# conditions = (aperes['afternoon'] > aperes['evening']) & \
#              (aperes['afternoon'] > 2 * aperes['morning']) & \
#              (aperes['peak_hr'].isin([15, 16, 17, 18, 19, 20, 21]))

# aperes['APE'] = conditions

# pdf = aperes[['date','APE','afternoon','morning','evening','peak_hr']]


# pdf.set_index('date', inplace=True)


# np.sum(pdf['APE'])

373

In [103]:
# conditions = (gaperes['afternoon'] > gaperes['evening']) & \
#              (gaperes['afternoon'] > 2 * gaperes['morning']) & \
#              (gaperes['peak_hr'].isin([15, 16, 17, 18, 19, 20, 21]))

# gaperes['APE'] = conditions

# pdf = gaperes[['date','APE','afternoon','morning','evening','peak_hr']]


# pdf.set_index('date', inplace=True)


# np.sum(pdf['APE'])

329

In [10]:
#deprecate -- does not shift time zones correctly

# p_per_hr = []

# for df in cleanp:
    
#     time = time_adjustment(df)
    
#     precip_totals = np.zeros(df.variables['precip'].shape[0])
    
#     hrs = df.variables['precip'].shape[0]
    
#     for hr in range(hrs):
    
#         for index in precip_index_of_pairs:
          
#             p = df.variables['precip'].data[hr,index[0],index[1]]
            
#             #convert to mm
            
#             p = p * 0.01
                
#             precip_totals[hr]+= p
    
#     avgp = precip_totals/len(precip_index_of_pairs)
    
#     tup = (time, avgp)
    
#     p_per_hr.append(tup)

# print(len(p_per_hr))

# avgpdf = pd.DataFrame(p_per_hr,columns=['date','precipitation'])

# avgpdf.index = avgpdf['date']

# avgpdf = avgpdf.drop('date',axis=1)

In [111]:
#hurdat removal of TC events
#replacing with gpdf

sgp = (36.6, -97.5)

tc_list = []

tc_dis_list = []

tc_time_list = []

for id_ in hurdat.keys:
    
    if int(id_[-4:]) < 2000:
        
        continue
    
    lat = np.array(hurdat.data[id_]['lat'][:])
    
    lon = np.array(hurdat.data[id_]['lon'][:])
    
    dis_list = []
    
    for j in range(len(lat)):
        
        pt = (lat[j],lon[j])
        
        dis_list = np.append(dis_list, haversine(sgp, pt, unit='km'))
    
    if np.any(dis_list < 800):
            
        tc_time_list = np.append(tc_time_list, np.array(hurdat.data[id_]['time'])[np.where(dis_list < 500)])
            
        tc_list.append(id_)


tc_time = []

for tt in tc_time_list:
    
    tc_time.append(np.datetime64(tt))   


ok_tc_time = []

for i in tc_time:
    
    ok_tc_time.append(i - pd.to_timedelta('6 hours'))
        
        
tc_dates = []

for dt in ok_tc_time:
    
    d = dt.date()
    
    tc_dates.append(d)


tc_dates = list(set(tc_dates))

gpdf_minus_hu = gpdf[~gpdf.index.isin(tc_dates)]

gpdf_only_hu = gpdf[gpdf.index.isin(tc_dates)]

print(gpdf_minus_hu.shape)

print("ape count", sum(gpdf_minus_hu['APE']==True))

(2718, 2)
ape count 354


In [112]:
with open('pdf_after_changes.pdkl', 'wb') as file:
    pickle.dump(pdf_minus_hu, file)